<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/SalesMailPlace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mecab-python3
!pip install mojimoji

     |████████████████████████████████| 15.9MB 2.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/67/f0/6e/03e5c5017afc8230e19abeacad57189138207fea2f5dd71880
Successfully built mojimoji


In [0]:
import re
import MeCab
import mojimoji

def serialize(words):
    #大文字統一
    #words = words.upper()

    #英数字のみ半角統一
    words = mojimoji.zen_to_han(words, kana=False)
      
    #改行コード変換
    #words = words.replace("\r\n","")

    return words

In [0]:
def tanka_pickup(str_wk):
  #======================================================
  #スキップワード削除
  #======================================================
  del_word = ["<",">","＜","＞","[","]","《","》","≪","≫","【", "】"," ","　",":","：","・","弊社","駅","直結","オフィス","スキル"]
  
  #路線名パターン
  line_ptn_list = ['JR([ぁ-ゟ]+|[\ァ-ヿ]+|[一-鿐]+)+',
            '東武([ぁ-ゟ]+|[\ァ-ヿ]+|[一-鿐]+)+',
            '西武([ぁ-ゟ]+|[\ァ-ヿ]+|[一-鿐]+)+',
            '([ぁ-ゟ]+|[\ァ-ヿ]+|[一-鿐]+)+線',
            '([ぁ-ゟ]+|[\ァ-ヿ]+|[一-鿐]+)+鉄道',
            '([ぁ-ゟ]+|[\ァ-ヿ]+|[一-鿐]+)+メトロ',
            '([ぁ-ゟ]+|[\ァ-ヿ]+|[一-鿐]+)+ライン'
            ]

  base_word = '場所'
  tanka_yuragi = ["最寄駅","作業場所","勤務地"]
  
  #ゆらぎ統一
  for tanka in tanka_yuragi:
    str_wk = str_wk.replace(tanka,base_word)

  #余分な文章を削除
  base_word_idx = str_wk.find(base_word);
  if base_word_idx > 0:
    str_wk = str_wk[base_word_idx:base_word_idx+20]
  else:
    str_wk = ''
  
  #路線名削除
  for line in line_ptn_list:
    del_line_word = re.search(line,str_wk)
    if del_line_word != None:
      str_wk = str_wk.replace(del_line_word.group(),"")
  
  #不要文字列削除
  for word in del_word:
    str_wk = str_wk.replace(word,'')
  
  #正規化
  str_wk = serialize(str_wk)
    
  #======================================================
  #分かち書き
  #======================================================
  tagger = MeCab.Tagger("-Owakati")
  list_output = (tagger.parse(str_wk)).split(' ')

  #======================================================
  #「場所」に最も近い位置で書かれている数字が含まれる文節を抽出
  #======================================================
  tanka_str = ""
    
  if list_output.count(base_word) > 0:
    if re.search('[ぁ-ゟ]+|[\ァ-ヿ]+|[一-鿐]+',list_output[list_output.index(base_word)+1]) != None:
      tanka_str = re.search('[ぁ-ゟ]+|[\ァ-ヿ]+|[一-鿐]+',list_output[list_output.index(base_word)+1])
      tanka_str = tanka_str.group()

      if re.search('[ぁ-ゟ]+|[\ァ-ヿ]+|[一-鿐]+',list_output[list_output.index(base_word)+2]) != None:
        tanka_str2 = re.search('[ぁ-ゟ]+|[\ァ-ヿ]+|[一-鿐]+',list_output[list_output.index(base_word)+2])
        tanka_str = tanka_str + str(tanka_str2.group())
    else:
      tanka_str = "勤務地情報あり"
  else:
    tanka_str = "勤務地情報なし"
    
  return tanka_str

In [19]:
import pandas as pd
import codecs

#utf-8で作られていないファイルの場合のCSV読み込み方法
with codecs.open("python.CSV", "r", "Shift-JIS", "ignore") as file:
    df = pd.read_table(file, delimiter=",")

df2 = df[["本文"]]

#行単位の繰り返し
for index,item in df2.iterrows():
  str_wk = item['本文']
  pickup_str = tanka_pickup(str_wk)
  print(str(index) + ":" + pickup_str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead.
  


0:勤務地情報なし
1:勤務地情報なし
2:勤務地情報なし
3:勤務地情報なし
4:神奈川県
5:神田
6:恵比寿
7:京橋
8:東陽町
9:恵比寿
10:東京
11:品川
12:浜松町
13:大手町
14:六本木
15:六本木
16:池袋
17:浜松町
18:東陽町
19:東京
20:池袋
21:恵比寿
22:神田
23:恵比寿
24:勤務地情報なし
25:調布
26:東陽町
27:勤務地情報なし
28:大手町
29:六本木
30:六本木
31:浜松町
32:浜松町
33:調布
34:大手町
35:勤務地情報なし
36:六本木
37:六本木
38:東陽町
39:浜松町
40:白金高輪
41:浜松町
42:白金高輪
43:勤務地情報なし
44:勤務地情報なし
45:勤務地情報なし
46:恵比寿
47:神田
48:勤務地情報なし
49:勤務地情報なし
50:京橋
51:恵比寿
52:東陽町
53:東京
54:大手町
55:六本木
56:六本木
57:東陽町
58:浜松町
59:浜松町
60:渋谷
61:神田
62:勤務地情報あり
63:東京
64:東陽町
65:神谷町
66:大手町
67:勤務地情報あり
68:池袋
69:浜松町
70:浜松町
71:東京
72:六本木
73:六本木
74:東陽町
75:調布
76:渋谷
77:神田
78:勤務地情報なし
79:東京
80:千葉ニュータウン
81:勤務地情報あり
82:東京
83:東陽町
84:勤務地情報なし
85:神田
86:勤務地情報なし
87:東京都
88:六本木
89:六本木
90:六本木
91:浜松町
92:浜松町
93:勤務地情報あり
94:大手町
95:勤務地情報なし
96:駿河台
97:渋谷
98:勤務地情報あり
99:勤務地情報なし
100:神田
101:大手町
102:渋谷
103:浜松町
104:東京都
105:勤務地情報なし
106:六本木
107:六本木
108:六本木
109:中野
110:大手町
111:浜松町
112:浜松町
113:六本木
114:六本木
115:勤務地情報あり
116:六本木
117:渋谷
118:中野
119:神田
120:勤務地情報あり
121:勤務地情報あり
122:六本木
123:六本木
124:浜松町
125:浜松町
126:大手町
127:八王子市
128: